<a target="_blank" href="https://colab.research.google.com/github/trainocate-japan/Machine-Learning-and-Deep-Learning-Hands-on/blob/main/exercise/5_勾配ブースティング木/5-2_（演習）LightGBMによるガンの悪性腫瘍の分類.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 5-2_（演習）LightGBMによるガンの悪性腫瘍の分類
こちらはロジスティック回帰で扱っているガンの悪性腫瘍判定と学習を開始するまで同じプログラムになっています。（LightGBM・交差検証・グリッドサーチ用のライブラリインポートは除く）

※ ただし、決定木系のアルゴリズムではスケールをそろえる必要がないため、ロジスティック回帰モデルで精度改善のために行っていた標準化は行いません。

## ライブラリのインポート

In [ ]:
# データを処理するための基本的なライブラリ
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
# LightGBMモデルをインポート
#★from lightgbm import LGBMClassifier

In [ ]:
# matplotlibで日本語表示するための設定
!pip install japanize_matplotlib | tail -n 1
import japanize_matplotlib

## データの準備
今回使用するデータはscikit-learnからもデータセットとして利用することができる、UCI ML Breast Cancer Wisconsin (Diagnostic) datasetsのコピーです。<br>
UCI Machine Learning Repositoryから公開されています。<br>
downloaded from : https://goo.gl/U2Uwz2

データセットについての説明はこちらに記載されています。<br>
https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset

#### データを取り込む
- pandasのread_csvメソッドを使用して、cancer.csvファイルを読み込みます
- 読み込んだものは変数df_cancerに代入します

In [ ]:
# ★★cancer.csv を変数df_cancerに読み込みます


#### データを確認する

In [ ]:
# ★★読み込んだデータをheadメソッドで確認します
# Classが目的変数になる腫瘍の悪性または良性を表しています。（0:悪性、1:良性）


In [ ]:
# ★★df_cancerのデータ要約を確認


In [ ]:
# ★★df_cancerの統計情報を確認


#### 説明変数、目的変数を切り出す

In [ ]:
# ★★dropメソッドを使って、目的変数にするClass以外をすべて説明変数xに代入する


In [ ]:
# ★★headメソッドでxの中身を確認する


In [ ]:
# ★★Classを目的変数yに代入する


In [ ]:
# ★★headメソッドでyの中身を確認する


#### データを訓練データと検証データに分割する

**本研修でbreast-cancerデータセットを使用する際には、訓練データ70%、random_state=3で固定しています。他のモデルとの比較をしやすくするためです**

In [ ]:
# ★★訓練データと検証データに分割(70%を訓練用に使用、random_state=3を指定、stratify=yを指定)
# stratifyは指定した列の値が均等に割り振られる


## モデルの定義

In [ ]:
# ★★変数modelにLGBMClassifierの機械学習モデルのインスタンスを格納する


## モデルの学習

In [ ]:
# ★★定義したモデルで学習をおこなう


## 評価

In [ ]:
# ★★訓練データで予測精度（正解率）を確認する


In [ ]:
# ★★検証データで予測精度（正解率）を確認する


## LightGBMのパラメータチューニング

In [ ]:
# ★★ハイパーパラメータを変えて、精度の変化を確認してみましょう。
#★model = LGBMClassifier(learning_rate=0.1, n_estimators=*, num_leaves=*, random_state=0)
#★model.fit(train_x, train_y)

In [ ]:
# 訓練データで予測精度（正解率）を確認する
#★model.score(train_x, train_y)

In [ ]:
# テストデータで予測精度（正解率）を確認する
#★model.score(val_x, val_y)

**※ 調整例**

`model = LGBMClassifier(learning_rate=0.1, n_estimators=60, num_leaves=8, random_state=0)`

97%程の精度が出ます。。

## 交差検証とグリッドサーチ

In [ ]:
# 交差検証とグリッドサーチ用のライブラリをインポート
#★from sklearn.model_selection import StratifiedKFold
#★from sklearn.model_selection import GridSearchCV

In [ ]:
# 探索したいパラメーターをディクショナリで設定する
# ★★ *の部分にパラメータのリストを設定しましょう。リストしたバリエーションに応じて時間がかかるため注意しましょう。
param_grid = {"learning_rate":[0.1],
              "num_leaves": [*],
              "colsample_bytree": [*],
              "subsample": [*],
              "n_estimators":[*]
             } 

# パラメータチューニングをグリッドサーチ
gscv = GridSearchCV(estimator = LGBMClassifier(random_state = 0), # パラメータ探索するモデル
                           param_grid = param_grid, # 探索するパラメータのリスト
                           cv = 4, # 交差検証のデータの分割数
                           verbose=2) # ログをどれだけ出力するか。進み具合のわかる2がおすすめ

gscv.fit(x, y)

In [ ]:
# スコアの一覧を取得
gs_result = pd.DataFrame.from_dict(gscv.cv_results_)
gs_result.sort_values('rank_test_score')[:3]

In [ ]:
# 最高性能のモデルを取得
#★best = gscv.best_estimator_
#★best

In [ ]:
# 参考に全データを用いて精度（正解率）を確認してみる
best.score(x, y)

In [ ]:
# classification_reportを確認してみる
#★prediction = best.predict(x)
#★print(classification_report(y, prediction))

## 特徴量（説明変数）の重要度を確認する

In [ ]:
# 特徴量重要度
#★importances= best.feature_importances_
#★df_importances =pd.DataFrame(data=importances, index=train_x.columns)
#★df_importances.sort_values(0, ascending=False)

In [ ]:
# （参考）matplotlibで画像サイズの調整（横幅、縦幅）
plt.figure(figsize=(10,12))
n_features = train_x.shape[1] # 特徴量の数
plt.barh(range(n_features), best.feature_importances_, align='center')
plt.yticks(np.arange(n_features), train_x.columns)
plt.plot;